In [114]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install scikit-learn
!{sys.executable} -m pip install torch
!{sys.executable} -m pip install rdkit


You should consider upgrading via the 'c:\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Python39\python.exe -m pip install --upgrade pip' command.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Python39\\Lib\\site-packages\\~cipy.libs\\libopenblas-802f9ed1179cb9c9b03d67ff79f48187.dll'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'c:\Python39\python.exe -m pip install --upgrade pip' command.



  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1


In [253]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd


In [254]:
# Define a Reaction dataset class
class ReactionDataset(Dataset):
    def __init__(self, X, y_G_act, y_G_r):
        self.X = X
        self.y_G_act = y_G_act
        self.y_G_r = y_G_r

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y_G_act[idx], self.y_G_r[idx]

In [255]:
# Read the dataset
df = pd.read_csv('full_dataset.csv') 

# Prepare the feature matrix X and target variables y
X_smiles = df['rxn_smiles']
y_G_act = df['G_act']
y_G_r = df['G_r']

In [256]:
# Define the encoding function
def encode_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)
    bitstring = fp.ToBitString()
    features = np.array([int(bit) for bit in bitstring], dtype=np.float32)
    features_tensor = torch.tensor(features)
    return features_tensor

# Encode SMILES strings to fingerprints
X_fingerprints = []

for idx, smiles in enumerate(X_smiles):
    reactants, products = smiles.split('>>')
    reactant_fingerprints = [encode_smiles(reactant) for reactant in reactants.split('.')]
    product_fingerprints = [encode_smiles(product) for product in products.split('.')]
    X_fingerprints.append(reactant_fingerprints + product_fingerprints)

# Convert the list of fingerprints and target values to tensors
X_tensor = torch.stack([torch.cat(fingerprints) for fingerprints in X_fingerprints])

# Print the fingerprint tensor
print(X_tensor)
print(X_tensor.shape)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
torch.Size([5269, 3072])


In [257]:
# Convert the data into PyTorch tensors
y_G_act_tensor = torch.tensor(y_G_act.values, dtype=torch.float32)
y_G_r_tensor = torch.tensor(y_G_r.values, dtype=torch.float32)

In [258]:
# Split the dataset into training and testing sets
X_train, X_test, y_G_act_train, y_G_act_test, y_G_r_train, y_G_r_test = train_test_split(X_tensor, y_G_act_tensor, y_G_r_tensor, test_size=0.2, random_state=42)


In [259]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(128, 128)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(128, 64)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(0.2)
        self.fc4_act = nn.Linear(64, 1)  # Output layer for G_act
        self.fc4_r = nn.Linear(64, 1)  # Output layer for G_r

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.dropout3(x)
        output_act = self.fc4_act(x)  # Output for G_act
        output_r = self.fc4_r(x)  # Output for G_r
        return output_act, output_r

In [260]:
# Instantiate the model
model = NeuralNetwork(X_train.shape[1])

In [261]:
# Define the loss function and optimizer
criterion = nn.SmoothL1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)

In [262]:
# Define the data loaders
train_dataset = ReactionDataset(X_train, y_G_act_train, y_G_r_train)
test_dataset = ReactionDataset(X_test, y_G_act_test, y_G_r_test)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32)

In [264]:
# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, targets_G_act, targets_G_r in train_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        outputs_G_act, outputs_G_r = outputs  # Separate the outputs into two tensors
        loss_G_act = criterion(outputs_G_act, targets_G_act.unsqueeze(1))
        loss_G_r = criterion(outputs_G_r, targets_G_r.unsqueeze(1))
        loss = loss_G_act + loss_G_r
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}")

Epoch 1/1000, Loss: 21.89886762156631
Epoch 2/1000, Loss: 13.191494486548685
Epoch 3/1000, Loss: 10.78838198112719
Epoch 4/1000, Loss: 9.930351156176943
Epoch 5/1000, Loss: 9.428149804924473
Epoch 6/1000, Loss: 8.972054615165248
Epoch 7/1000, Loss: 8.739297231038412
Epoch 8/1000, Loss: 8.557795336752227
Epoch 9/1000, Loss: 8.184347008213852
Epoch 10/1000, Loss: 8.142174930283517
Epoch 11/1000, Loss: 7.974739851373615
Epoch 12/1000, Loss: 7.828078277183302
Epoch 13/1000, Loss: 7.63043111743349
Epoch 14/1000, Loss: 7.7647493644194165
Epoch 15/1000, Loss: 7.532644712563717
Epoch 16/1000, Loss: 7.481084960879701
Epoch 17/1000, Loss: 7.369630354823488
Epoch 18/1000, Loss: 7.262411554654439
Epoch 19/1000, Loss: 7.217942906148506
Epoch 20/1000, Loss: 7.214967398932486
Epoch 21/1000, Loss: 7.1029068159334585
Epoch 22/1000, Loss: 7.050482005784006
Epoch 23/1000, Loss: 6.865694060470119
Epoch 24/1000, Loss: 6.910609678788618
Epoch 25/1000, Loss: 6.806640531077529
Epoch 26/1000, Loss: 6.894545193

In [265]:
# Evaluate the model on the testing data
model.eval()
with torch.no_grad():
    test_loss_G_act = 0.0
    test_loss_G_r = 0.0

    for inputs, targets_G_act, targets_G_r in test_dataloader:
        outputs = model(inputs)
        outputs_G_act, outputs_G_r = outputs  # Separate the outputs into two tensors

        loss_G_act = criterion(outputs_G_act, targets_G_act.unsqueeze(1))
        loss_G_r = criterion(outputs_G_r, targets_G_r.unsqueeze(1))

        test_loss_G_act += loss_G_act.item()
        test_loss_G_r += loss_G_r.item()

        print(f"outputs G_act: {outputs_G_act}")
        print(f"outputs G_r: {outputs_G_r}")

    average_test_loss_G_act = test_loss_G_act / len(test_dataloader)
    average_test_loss_G_r = test_loss_G_r / len(test_dataloader)

    print(f"Average Test Loss G_act: {average_test_loss_G_act}")
    print(f"Average Test Loss G_r: {average_test_loss_G_r}")

outputs G_act: tensor([[18.1612],
        [ 7.4811],
        [ 5.3078],
        [14.2548],
        [23.5206],
        [25.5965],
        [11.8800],
        [25.8802],
        [30.9657],
        [30.2225],
        [16.0195],
        [10.9047],
        [18.0491],
        [20.3994],
        [18.1849],
        [30.7277],
        [19.3494],
        [27.4446],
        [26.6009],
        [24.1394],
        [34.4497],
        [13.0333],
        [16.5097],
        [27.5808],
        [26.4234],
        [20.3017],
        [31.2845],
        [32.1672],
        [28.1824],
        [18.6490],
        [16.6567],
        [12.9035]])
outputs G_r: tensor([[ -0.7273],
        [-78.1272],
        [-46.4466],
        [-44.0128],
        [-26.5577],
        [-16.2092],
        [-39.3486],
        [ -6.0317],
        [  3.4190],
        [  3.0248],
        [-40.0658],
        [-73.6674],
        [-29.5666],
        [-16.4575],
        [-38.7464],
        [ -3.2144],
        [-22.6353],
        [ -2.7243],
   

# For showing how it works, we will use the following example:

In [271]:

# Read the dataset
df = pd.read_csv('debuging_dataset.csv')

# Extract the SMILES strings
smiles = df['rxn_smiles']

print(f"G_act {df['G_act']} G_r {df['G_r']}  ")

# Encode SMILES strings to fingerprints
X_fingerprints = []

for idx, smiles in enumerate(smiles):
    reactants, products = smiles.split('>>')
    reactant_fingerprints = [encode_smiles(reactant) for reactant in reactants.split('.')]
    product_fingerprints = [encode_smiles(product) for product in products.split('.')]
    X_fingerprints.append(reactant_fingerprints + product_fingerprints)

# Convert the list of fingerprints and target values to tensors
X_test = torch.stack([torch.cat(fingerprints) for fingerprints in X_fingerprints])


G_act 0     15.875896
1     15.155477
2     18.013824
3     23.687079
4     23.438094
5     20.969801
6     11.722625
7      7.268044
8      6.289673
9     16.044475
10    13.604005
Name: G_act, dtype: float64 G_r 0    -51.881526
1    -51.398681
2    -66.822349
3    -62.481289
4    -84.836459
5    -57.785046
6    -63.721331
7    -86.996102
8    -86.799671
9    -50.409197
10   -51.008126
Name: G_r, dtype: float64  


In [273]:

# Set the model to evaluation mode
model.eval()

# Make predictions on test inputs
with torch.no_grad():
    test_inputs = X_test  
    predictions_G_act, predictions_G_r = model(test_inputs)  # Separate the predictions into two tensors

# Convert predictions to numpy arrays
predictions_G_act = predictions_G_act.numpy()
predictions_G_r = predictions_G_r.numpy()


print("Predictions for G_act:")
for pred in predictions_G_act:
    print(pred[0])

print("Predictions for G_r:")
for pred in predictions_G_r:
    print(pred[0])

Predictions for G_act:
14.894965
15.17654
18.077942
23.272684
21.9313
19.457027
11.238342
6.3206444
7.481103
15.930947
13.589463
Predictions for G_r:
-50.983437
-50.43715
-65.683136
-61.65767
-82.97296
-57.48965
-64.0065
-86.8749
-78.12715
-49.251316
-50.61
